# Simulation of the plant (reflow oven) and the controller

In this notebook we simulate both the plant along with the controller.

## System identification of the plant

First of all, we need to have a sufficiently accurate model of the plant.

Previous tests showed that it can be modelled as two first oder lags and one delay in series.<br />
This models the (relatively fast) heating process and the slow cooling process, as well as some
delay.

A more accurate physical model shows that we have four first order lags and one delay.

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
import pandas as pd

In [41]:
# 100% PWM signal with stop after 100 deg C (October)
path_prefix = './../../measurements/2024-10-27/'
meas_file = 'signals_step_100_percent_until_100_degC_cooling_down_door_closed.log'
signal_100_deg_C_stop = path_prefix + meas_file

# 100% PWM signal with stop after 200 deg C (October)
path_prefix = './../../measurements/2024-10-27/'
meas_file = 'signals_step_100_percent_until_200_degC_cooling_down_door_closed.log'
signal_200_deg_C_stop = path_prefix + meas_file

df = pd.read_csv(signal_200_deg_C_stop, skiprows=2)

In [ ]:
fig = plt.figure(meas_file, figsize=(10,5))
plt.clf()

ax = fig.add_subplot(1,1,1)
ax.plot(df['Time (ms)']/1000, df['Oven temperature (C)'],
         linestyle='-', label='Oven temperature (C)')
ax.plot(df['Time (ms)']/1000, df['Ambient temperature (C)'],
         linestyle='--', label='Ambient temperature (C)')
ax.plot(df['Time (ms)']/1000, df['PWM controller (percent)'],
         linestyle='--', label='PWM controller (percent)')
ax.plot(df['Time (ms)']/1000, df['Thermocouple open']*25,
         linestyle='--', label='Thermocouple open (x25)')
ax.grid(True)
ax.set(ylim=(0, 300))
ax.set(xlabel='t (ms)')
ax.set(title='Measured signals')
ax.legend()

plt.show()

### Simple 2nd order model (with delay)

$$G_1(s) = \frac{6.7}{30s + 1}$$

$$G_2(s) = \frac{1.0}{480s + 1}$$

$$G_3(s) = \exp(-22 s)$$

In [43]:
# Simple model of the reflow oven fitted to the 100% PWM step that turns off once 100 deg C are reached.
# The parameters of the model have been obtained by manual tuning (not by numerical optimization).
tf_pt1_a = ctrl.tf([6.7], [30.0, 1.0])    # <-- Heating process with 30 seconds time constant
tf_pt1_b = ctrl.tf([1.0], [480.0, 1.0])   # <-- Cooling process with 480 seconds time constants
num, den = ctrl.pade(22, 5)               # <-- 22 seconds delay
tf_Td = ctrl.tf(num, den)
G = ctrl.series(tf_Td, tf_pt1_a, tf_pt1_b)

In [ ]:
# Read the first 2 lines of the log file.
# There we find the sample time and the maximum runtime.
with open(signal_200_deg_C_stop) as fid:
    first_lines = [l for idx, l in enumerate(fid.readlines()) if idx <= 1]

sample_time_ms = int(first_lines[0].strip('Sample time (ms): '))
max_runtime_seconds = int(first_lines[1].strip('Maximum runtime (s): '))

print(f'Sample time: {sample_time_ms} ms')
print(f'Maximum runtime: {max_runtime_seconds} seconds')


Model as linear system using the linear systems functions.

In [ ]:
# Make sure t is equally spaced
sample_time_seconds = sample_time_ms * 1.e-3
sample_frequency = 1.0 / sample_time_seconds
t = np.arange(0,max_runtime_seconds*sample_frequency+1) * sample_time_seconds

theta = df['Oven temperature (C)'].to_numpy()
theta_0 = np.mean(theta[:40])
print(f'theta_0 = {theta_0} deg C')

u = df['PWM controller (percent)'].to_numpy()

time_response = ctrl.forced_response(G, t, u, 0.0)

fig = plt.figure('simulation', figsize=(10,5))
plt.clf()
ax = fig.add_subplot(1,1,1)

ax.plot(t, u, linestyle='--', label='PWM (percent)')
ax.plot(t, theta - theta_0, linestyle='-', label='Meas. temperature (minus offset)')
ax.plot(t, time_response.outputs, linestyle='-', label='Sim. temperature (minus offset)')
ax.legend()
ax.grid(True)
ax.set(title='Simulation with simple model')

plt.show()

Model as (potentially) nonlinear system.

Also, consider the ambient temperature, that is, do not remove the constant offset in the oven temperature like before.

In [ ]:
#
# Oven dynamics
#
def oven_update_fun(t, x, u, params):
    theta_o_k = x[0] # <-- We have one state, the oven temperature
    theta_h_k = u[0] # <-- The first input is the actual input: the heating temperature
    theta_a_k = u[1] # <-- The second input is the disturbance: the ambient temperature
    q_zu_k = params['c']*(theta_h_k - theta_o_k)
    q_ab_o_k = params['alpha']*(theta_o_k - theta_a_k)
    dt = params['dt']
    theta_o_kp1 = theta_o_k + dt*(q_zu_k - q_ab_o_k)
    return theta_o_kp1

def oven_output_fun(t, x, u, params):
    theta_o_k = x[0]
    return theta_o_k

oven_params = { 'c': 0.5,
                'alpha': 0.5,
                 'dt': sample_time_seconds }

H_oven = ctrl.NonlinearIOSystem(
    oven_update_fun,
    oven_output_fun,
    inputs=['theta_h', 'theta_a'],
    outputs=['theta_o'],
    states=1,
    dt=oven_params['dt'],
    name='Oven',
    params=oven_params)

print(H_oven)

#
# Heating element dynamics
#
def heating_elem_update_fun(t, x, u, params):
    theta_h_k = x[0] # <-- We have one state, the heating element temperature
    u_k = u[0]       # <-- The first input is the actual input: the control input
    theta_a_k = u[1] # <-- The second input is the disturbance: the ambient temperature
    q_ab_h_k = params['beta']*(theta_h_k - theta_a_k)
    dt = params['dt']
    theta_h_kp1 = theta_h_k + dt*(params['K']*u_k - q_ab_h_k)
    return theta_h_kp1

def heating_elem_output_fun(t, x, u, params):
    theta_h_k = x[0]
    return theta_h_k

heating_elem_params = { 'K': 1/480.0,
                        'beta': 1/480.0,
                        'dt': sample_time_seconds }

H_heating_elem = ctrl.NonlinearIOSystem(
    heating_elem_update_fun,
    heating_elem_output_fun,
    inputs=['u_pwm', 'theta_a'],
    outputs=['theta_h'],
    states=1,
    dt=heating_elem_params['dt'],
    name='HeatingElement',
    params=heating_elem_params)

print(H_heating_elem)

#
# Time delay
#
H_delay_num = [1.0]
H_delay_den = np.zeros((22*int(sample_frequency)+1,))
H_delay_den[0] = 1.0
H_delay = ctrl.tf(H_delay_num, H_delay_den, sample_time_seconds, inputs='u', outputs='y')
print(H_delay)
H_delay = ctrl.ss(H_delay, name='TimeDelay')



In [ ]:
#
# Connect the systems
#
H_connected = ctrl.interconnect(
    [H_oven, H_heating_elem, H_delay],
    connections=[['TimeDelay.u', 'HeatingElement.theta_h'],
                 ['Oven.theta_h', 'TimeDelay.y']
                ],
    inplist=['HeatingElement.u_pwm',
             ['Oven.theta_a',
             'HeatingElement.theta_a']
             ],
    outlist=['Oven.theta_o'],
    inputs=['u_pwm_overall', 'theta_a_overall'],
    outputs=['theta_o_overall'],
    dt=sample_time_seconds
)

print(H_connected)

In [ ]:
# Make sure t is equally spaced
sample_time_seconds = sample_time_ms * 1.e-3
sample_frequency = 1.0 / sample_time_seconds
t_input = np.arange(0,max_runtime_seconds*sample_frequency+1) * sample_time_seconds

theta_o = df['Oven temperature (C)'].to_numpy()

u_pwm_input = df['PWM controller (percent)'].to_numpy()
theta_a_input = df['Ambient temperature (C)'].to_numpy()

u_input = np.vstack((u_pwm_input, theta_a_input))

print(t_input.shape)
print(u_input.shape)

X0 = np.hstack(([theta_o[0], theta_o[0]], theta_o[0]*np.ones((22*int(sample_frequency),))))
print(X0.shape)

results = ctrl.input_output_response(H_connected, t_input, u_input, X0=X0)

fig = plt.figure(1)
plt.clf()
# plt.plot(t_input, u_input, label='u')
plt.plot(results.t, results.y[0,:], label='theta_o')
plt.grid(True)
plt.legend()
plt.show()